In [ ]:
# default_exp layers

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from torch.nn.utils import weight_norm, spectral_norm

# Layers
> Custom fastai layers and basic functions to grab them.

## Basic manipulations and resize

In [ ]:
# export
class Lambda(nn.Module):
    "An easy way to create a pytorch layer for a simple `func`"
    def __init__(self, func):
        super().__init__()
        self.func=func

    def forward(self, x): return self.func(x)

> Warning: In the tests below, we use lambda functions for convenience, but you shouldn't do this when building a real modules as it would make models that won't pickle (so you won't be able to save/export them).

In [ ]:
tst = Lambda(lambda x:x+2)
x = torch.randn(10,20)
test_eq(tst(x), x+2)

In [ ]:
# export
class View(nn.Module):
    "Reshape `x` to `size`"
    def __init__(self, *size):
        super().__init__()
        self.size = size

    def forward(self, x): return x.view(self.size)

In [ ]:
tst = View(10,5,4)
test_eq(tst(x).shape, [10,5,4])

In [ ]:
# export
class ResizeBatch(nn.Module):
    "Reshape `x` to `size`, keeping batch dim the same size"
    def __init__(self, *size):
        super().__init__()
        self.size = size

    def forward(self, x):
        size = (x.size(0),) + self.size
        return x.view(size)

In [ ]:
tst = ResizeBatch(5,4)
test_eq(tst(x).shape, [10,5,4])

In [ ]:
# export
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

In [ ]:
tst = Flatten()
x = torch.randn(10,5,4)
test_eq(tst(x).shape, [10,20])
tst = Flatten(full=True)
test_eq(tst(x).shape, [200])

In [ ]:
class PoolFlatten(nn.Sequential):
    "Combine `nn.AdaptiveAvgPool2d` and `Flatten`."
    def __init__(self): super().__init__(nn.AdaptiveAvgPool2d(1), Flatten())

In [ ]:
tst = PoolFlatten()
x = torch.randn(10,5,4,4)
test_eq(tst(x).shape, [10,5])
test_eq(tst(x), x.mean(dim=[2,3]))

## Convolutions

In [ ]:
#export
def init_default(m, func=nn.init.kaiming_normal_):
    "Initialize `m` weights with `func` and set `bias` to 0."
    if func:
        if hasattr(m, 'weight'): func(m.weight)
        if hasattr(m, 'bias') and hasattr(m.bias, 'data'): m.bias.data.fill_(0.)
    return m

In [ ]:
def _relu(inplace:bool=False, leaky:float=None):
    "Return a relu activation, maybe `leaky` and `inplace`."
    return nn.LeakyReLU(inplace=inplace, negative_slope=leaky) if leaky is not None else nn.ReLU(inplace=inplace)

class ConvLayer(nn.Sequential):
    "Create a sequence of convolutional (`ni` to `nf`), ReLU (if `use_activ`) and `norm_type` layers."
    def __init__(self, ni, nf, ks=3, stride=1, padding=None, bias=None, is_1d=False, norm_type=NormType.Batch,
                 use_activ=True, leaky=None, transpose=False, init=nn.init.kaiming_normal_, xtra=None):
        if padding is None: padding = ((ks-1)//2 if not transpose else 0)
        bn = norm_type in (NormType.Batch, NormType.BatchZero)
        if bias is None: bias = not bn
        conv_func = nn.ConvTranspose2d if transpose else nn.Conv1d if is_1d else nn.Conv2d
        conv = init_default(conv_func(ni, nf, kernel_size=ks, bias=bias, stride=stride, padding=padding), init)
        if   norm_type==NormType.Weight:   conv = weight_norm(conv)
        elif norm_type==NormType.Spectral: conv = spectral_norm(conv)
        layers = [conv]
        if use_activ: layers.append(_relu(True, leaky=leaky))
        if bn: layers.append(nn.BatchNorm1d(nf) if is_1d else BatchNorm2d(nf, norm_type=norm_type))
        if xtra: layers.append(xtra)
        super().__init__(*layers)

The convolution uses `ks` (kernel size) `stride`, `padding` and `bias`. `padding` will default to the appropriate value (`(ks-1)//2` if it's not a transposed conv) and `bias` will default to `True` the `norm_type` is `No`, `Spectral` or `Weight`, `False` if it's `Batch` or `BatchZero`.

If `is_1d=True`, this defines a `Conv1d` layer, otherwise it uses a `Conv2d` if `transpose=False`, `ConvTranspose2d` if `transpose=True`. If `use_activ=True` an activation is added (before the potential BatchNorm) and is a `ReLU`, `leaky` if this argument is used. 

`init` is used to initialize the weights (the bias are initiliazed to 0) and `xtra` is an optional layer to add at the end.

In [ ]:
tst = ConvLayer(16, 32)
mods = list(tst.children())
test_eq(len(mods), 3)
assert isinstance(mods[0], nn.Conv2d)
assert isinstance(mods[1], nn.ReLU)
assert isinstance(mods[2], nn.BatchNorm2d)
test_eq(mods[2].weight, torch.ones(32))
test_eq(mods[0].padding, (1,1))
x = torch.randn(64, 16, 8, 8)
#Padding is selected to make the shape the same if stride=1
test_eq(tst(x).shape, [64,32,8,8])

#Padding is selected to make the shape half if stride=2
tst = ConvLayer(16, 32, stride=2)
test_eq(tst(x).shape, [64,32,4,4])

#You can laways pass your own padding if you want
tst = ConvLayer(16, 32, padding=0)
test_eq(tst(x).shape, [64,32,6,6])

In [ ]:
mods[0].padding

(1, 1)

## BatchNorm layers

In [ ]:
# export
NormType = Enum('NormType', 'Batch BatchZero Weight Spectral')

In [ ]:
#export
class BatchNorm2d(nn.BatchNorm2d):
    "A batchnorm2d layer with `nf` features initialized depending on `norm_type`."
    def __init__(self, nf, norm_type=NormType.Batch, **kwargs):
        super().__init__(nf, **kwargs)
        self.bias.data.fill_(1e-3)
        self.weight.data.fill_(0. if norm_type==NormType.BatchZero else 1.)

`kwargs` are passed to `nn.BatchNorm2d` and can be `eps`, `momentum`, `affine` and `track_running_stats`.

In [ ]:
tst = BatchNorm2d(15)
assert isinstance(tst, nn.BatchNorm2d)
test_eq(tst.weight, torch.ones(15))
tst = BatchNorm2d(15, norm_type=NormType.BatchZero)
test_eq(tst.weight, torch.zeros(15))

In [ ]:
class BnDropLin(nn.Sequential):
    "Module grouping `BatchNorm1d`, `Dropout` and `Linear` layers"
    def __init__(self, n_in, n_out, bn=True, p=0., act=None):
        layers = [nn.BatchNorm1d(n_in)] if bn else []
        if p != 0: layers.append(nn.Dropout(p))
        layers.append(nn.Linear(n_in, n_out))
        if act is not None: layers.append(act)
        super().__init__(*layers)

The `BatchNorm` layer is skipped if `bn=False`, as is the dropout if `p=0.`. Optionally, you can add an activation for after the linear laeyr with `act`.

In [ ]:
tst = BnDropLin(10, 20)
mods = list(tst.children())
test_eq(len(mods), 2)
assert isinstance(mods[0], nn.BatchNorm1d)
assert isinstance(mods[1], nn.Linear)

tst = BnDropLin(10, 20, p=0.1)
mods = list(tst.children())
test_eq(len(mods), 3)
assert isinstance(mods[0], nn.BatchNorm1d)
assert isinstance(mods[1], nn.Dropout)
assert isinstance(mods[2], nn.Linear)

tst = BnDropLin(10, 20, act=nn.ReLU())
mods = list(tst.children())
test_eq(len(mods), 3)
assert isinstance(mods[0], nn.BatchNorm1d)
assert isinstance(mods[1], nn.Linear)
assert isinstance(mods[2], nn.ReLU)

tst = BnDropLin(10, 20, bn=False)
mods = list(tst.children())
test_eq(len(mods), 1)
assert isinstance(mods[0], nn.Linear)

## Self attention

In [ ]:
class PooledSelfAttention2d(nn.Module):
    "Pooled self attention layer for 2d."
    def __init__(self, n_channels:int):
        super().__init__()
        self.theta = spectral_norm(conv2d(n_channels, n_channels//8, 1))
        self.phi   = spectral_norm(conv2d(n_channels, n_channels//8, 1))
        self.g     = spectral_norm(conv2d(n_channels, n_channels//2, 1))
        self.o     = spectral_norm(conv2d(n_channels//2, n_channels, 1))
        self.gamma = nn.Parameter(tensor([0.]))

    def forward(self, x):
        # code borrowed from https://github.com/ajbrock/BigGAN-PyTorch/blob/7b65e82d058bfe035fc4e299f322a1f83993e04c/layers.py#L156
        theta = self.theta(x)
        phi = F.max_pool2d(self.phi(x), [2,2])
        g =   F.max_pool2d(self.g(x),   [2,2])    
        theta = theta.view(-1, self. ch//8, x.shape[2]*x.shape[3])
        phi   = phi  .view(-1, self. ch//8, x.shape[2]*x.shape[3]//4)
        g     = g    .view(-1, self. ch//2, x.shape[2]*x.shape[3]//4)
        beta = F.softmax(torch.bmm(theta.transpose(1, 2), phi), -1)
        o = self.o(torch.bmm(g, beta.transpose(1,2)).view(-1, self.ch//2, x.shape[2], x.shape[3]))
        return self.gamma * o + x

In [ ]:
class SelfAttention(nn.Module):
    "Self attention layer for nd."
    def __init__(self, n_channels:int):
        super().__init__()
        self.query = ConvLayer(n_channels, n_channels//8, is_1d=True, norm_type=NormType.Spectral, use_activ=False, bias=False)
        self.key   = conv1d(n_channels, n_channels//8)
        self.value = conv1d(n_channels, n_channels)
        self.gamma = nn.Parameter(tensor([0.]))

    def forward(self, x):
        #Notation from https://arxiv.org/pdf/1805.08318.pdf
        size = x.size()
        x = x.view(*size[:2],-1)
        f,g,h = self.query(x),self.key(x),self.value(x)
        beta = F.softmax(torch.bmm(f.permute(0,2,1).contiguous(), g), dim=1)
        o = self.gamma * torch.bmm(h, beta) + x
        return o.view(*size).contiguous()